In [3]:
"""
Raw_data
"""
import pandas
import numpy
#from scipy.stats import mode
from sklearn import neighbors
from sklearn.neighbors import DistanceMetric 
from pprint import pprint

T_TRAIN = 'raw_data/train_titanic.csv'
T_TEST = 'raw_data/test_titanic.csv'

titanic_train = pandas.read_csv(T_TRAIN, header=0)

Describe the data.
> How big?
> What are the columns and what do they mean?

In [18]:
print('Length:\n\t{0} rows'.format(len(titanic_train)))
print('Columns:\n\t{0}'.format(titanic_train.columns))

Length:
	891 rows
Columns:
	Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


What’s the average age of...
> any Titanic passenger
> a survivor
> a non-surviving first-class passenger
> Male survivors older than 30 from anywhere but Queenstown
> For the groups you chose, how far (in years) are the average ages from the median ages?

In [81]:
avg_age_all = titanic_train['Age'].mean()
print('Average Age: Passenger\n\t{0}'.format(avg_age_all))

ave_age_survivor = titanic_train.loc[titanic_train['Survived'] == 1]['Age'].mean()
print('Average Age: Survivor\n\t{0}'.format(ave_age_survivor))

# Non-Survivor
ns = titanic_train.loc[titanic_train['Survived'] == 0]
# Non-Survivor First Class
ns_fc = ns.loc[ns['Pclass'] == 1] 
# Average Age of Non-Survivor First Class
ave_age_of_ns_fc = ns_fc["Age"].mean()
print('Average Age: Non-Survivor, First-class\n\t{0}'.format(ave_age_of_ns_fc))

survivors = titanic_train.loc[
    (titanic_train['Survived'] == 1) &
    (titanic_train['Sex'] == 'male') &
    (titanic_train['Age'] > 30) &
    (titanic_train['Embarked'] != 'Q')]
avg_age_not_q_male_survivor_over_30= survivors['Age'].mean()
print('Average Age: Male Survivor, > 30, not Queenstown\n\t{0}'.format(avg_age_not_q_male_survivor_over_30))

ages = titanic_train['Age'][1]
median_age = numpy.median(ages)
print('Median Age:\n\t{0}'.format(median_age))
print('Average from median Age: \n\t{0}'.format(median_age - avg_age_all))

Average Age: Passenger
	29.69911764705882
Average Age: Survivor
	28.343689655172415
Average Age: Non-Survivor, First-class
	43.6953125
Average Age: Male Survivor, > 30, not Queenstown
	41.48780487804878
Median Age:
	38.0
Average from median Age: 
	8.30088235294118


What’s the most common...
> passenger class
> port of Embarkation
> number of siblings or spouses aboard for survivors

> Within what range of standard deviations from the mean (0-1, 1-2, 2-3) is the median ticket price?
> Is it above or below the mean?

> How much more expensive was the 90th percentile ticket than the 5th percentile ticket? Are they the same class?
> The highest average ticket price was paid by passengers from which port? Null ports don’t count

> Which port has passengers from the most similar passenger class?




> Display the relationship (i.e. make a plot) between survival rate and the quantile of the ticket price for 20 integer quantiles. Make sure you label your axes.

>  What fraction of surviving 1st-class males paid lower than the overall median ticket price?

In [27]:
overall_fare = titanic_train['Fare'][1]
overall_median = numpy.median(overall_fare)
print(overall_median)
male_lower_median = titanic_train.loc[(titanic_train['Pclass'] == 1)&
                                      (titanic_train['Survived'] == 1)&
                                      (titanic_train['Sex'] == 'male')&
                                      (titanic_train['Fare'] < overall_median)]
lower_count = male_lower_median.count()
print(lower_no[0])
# take id to count total number
all_1st_class_male_survivor = titanic_train.loc[(titanic_train['Pclass'] == 1)&
                                      (titanic_train['Survived'] == 1)&
                                      (titanic_train['Sex'] == 'male')]
total_count = all_1st_class_male_survivor.count()
print(total_count[0])
                                
print("Fractionlower_count[0]/total_count[0])


71.2833
31
45
0.688888888889


>  How much older/younger was the average surviving passenger with family members than the average non-surviving passenger without them?

In [9]:
survive_w_sibsp_parch = titanic_train.loc[(titanic_train['Survived'] == 1)&
                                       (titanic_train['SibSp'] > 0)&
                                       (titanic_train['Parch'] > 0)] 
avg_age_survivor_family = survive_w_sibsp_parch['Age'].mean()
print(avg_age_survivor_family)
non_survivor_wo_family = titanic_train.loc[(titanic_train['Survived'] == 0)&
                                           (titanic_train['SibSp'] == 0)&
                                           (titanic_train['Parch'] == 0)]
avg_age_non_survivor_wo_family = non_survivor_wo_family['Age'].mean()
print(avg_age_non_survivor_wo_family)
diff = abs(avg_age_survivor_family - avg_age_non_survivor_wo_family)
print(diff)

20.71050847457627
32.41423357664234
11.70372510206607


*STRETCH GOAL* 
For each of the following characteristics, find the median in the data.
> age
> ticket price
> siblings/spouses
> parents/children

If you were to use these medians to draw numerical boundaries separating survivors from non-survivors, which of these characteristics would be the best choice and why?